In [1]:
import numpy as np
import model
import pandas as pd
import simpy

Example model, manufacturing system:

In [2]:
def bottle_filling(fbd,fsbd,lbd,lsbd,mo,frt,fsrt,lrt,lsrt,mt,p,mh):

    import numpy as np
    import simpy

        # Function to generate random breakdown times
    def breakdown_time():
        return np.random.exponential(fbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,fbd)  # Breakdown occurence distribution

    def small_breakdown_time():
        return np.random.exponential(fsbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,fsbd)  # Breakdown occurence distribution

    # Function to generate random breakdown times
    def labeller_breakdown_time():
        return np.random.exponential(lbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,lbd)  # Breakdown occurence distribution

    def labeller_small_breakdown_time():
        return np.random.exponential(lsbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,lsbd)  # Breakdown occurence distribution

    def maintenance_occurence_time():
        return mo  #Maintenance occurence distribution

    # Function to generate random repair times
    def repair_time():
        return np.random.exponential(frt)  #filler Repair time distribution
    def small_repair_time():
        return np.random.exponential(fsrt)  #filler small Repair time distribution

    def labeller_repair_time():
        return np.random.exponential(lrt)  #labeller Repair time distribution
    def labeller_small_repair_time():
        return np.random.exponential(lsrt)   #labeller small Repair time distribution
    def maintenance_time():
        return mt  # maintenance time

    # Bottling process
    def bottling(env, machine, stats):
        while True:
            yield env.timeout(1)  # Bottling takes 1 time unit

            if not machine.working:
                # If machine is not working due to breakdown, skip this cycle
                continue
            stats['total_bottles'] += 1

    # Breakdown process
    def breakdown(env, machine, stats):
        while True:
            yield env.timeout(breakdown_time())
            if machine.working:
                machine.working = False
                stats['filler_big_breakdowns'] += 1
                machine.down.succeed()  # Trigger the repair process
    def small_breakdown(env, machine, stats):
        while True:
            yield env.timeout(small_breakdown_time())
            if machine.working:
                machine.working = False
                stats['filler_small_breakdowns'] += 1
                machine.down2.succeed()  # Trigger the repair process
    # Breakdown process
    def labeller_breakdown(env, machine, stats):
        while True:
            yield env.timeout(labeller_breakdown_time())
            if machine.working:
                machine.working = False
                stats['labeller_big_breakdowns'] += 1
                machine.down3.succeed()  # Trigger the repair process
    def labeller_small_breakdown(env, machine, stats):
        while True:
            yield env.timeout(labeller_small_breakdown_time())
            if machine.working:
                machine.working = False
                stats['labeller_small_breakdowns'] += 1
                machine.down4.succeed()  # Trigger the repair process
    def maintenance(env, machine, stats):
        while True:
            yield env.timeout(maintenance_occurence_time())
            if machine.working:
                machine.working = False
                stats['Total maintenance occurences'] += 1
                machine.down5.succeed()  # Trigger the repair process

        # Repair process
    def repair(env, machine, stats):
        while True:
            yield machine.down
            repair_duration = repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['big_filler_downtime'] += repair_duration
            machine.working = True
            machine.down = env.event()  # Reset repair event

    def small_repair(env, machine, stats):
        while True:
            yield machine.down2
            repair_duration = small_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['small_filler_downtime'] += repair_duration
            machine.working = True
            machine.down2 = env.event()  # Reset repair event

    # Repair process
    def labeller_repair(env, machine, stats):
        while True:
            yield machine.down3
            repair_duration = labeller_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['labeller_big_downtime'] += repair_duration
            machine.working = True
            machine.down3 = env.event()  # Reset repair event

    def labeller_small_repair(env, machine, stats):
        while True:
            yield machine.down4
            repair_duration = labeller_small_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['labeller_small_downtime'] += repair_duration
            machine.working = True
            machine.down4 = env.event()  # Reset repair event

    def maintenance_action(env, machine, stats):
        while True:
            yield machine.down5
            repair_duration = maintenance_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['Total_maintenance_time'] += repair_duration
            machine.working = True
            machine.down5 = env.event()  # Reset repair event

    # Simulation setup
    env = simpy.Environment()
    machine = simpy.Resource(env, capacity=1)  # Bottling machine resource
    machine.working = True  # Initially machine is working
    machine.down = env.event()  # Create event for filler breakdown
    machine.down2 = env.event()  # Create event for small filler breakdown
    machine.down3 = env.event()  # Create event for labeller breakdown
    machine.down4 = env.event()  # Create event for small labeller breakdown
    machine.down5 = env.event()  # Create event for machine maintenance

    stats = {'total_bottles': 0, 'filler_big_breakdowns': 0, 'total_downtime': 0, 'filler_small_breakdowns': 0,
             'labeller_small_breakdowns': 0, 'labeller_big_breakdowns': 0 ,'small_filler_downtime': 0, 'big_filler_downtime': 0
            , 'Total_maintenance_time': 0,'labeller_small_downtime':0,'labeller_big_downtime':0,'Total maintenance occurences':0}

    # Start processes
    env.process(bottling(env, machine, stats))

    #Filler inputs:
    env.process(repair(env, machine, stats))
    env.process(breakdown(env, machine, stats))
    env.process(small_repair(env, machine, stats))
    env.process(small_breakdown(env, machine, stats))
    #Labeller inputs:
    env.process(labeller_repair(env, machine, stats))
    env.process(labeller_breakdown(env, machine, stats))
    env.process(labeller_small_repair(env, machine, stats))
    env.process(labeller_small_breakdown(env, machine, stats))
    #Maintenance inputs:
    env.process(maintenance_action(env, machine, stats))
    env.process(maintenance(env, machine, stats))

    # Run simulation
    simulation_time = 168*60*6 #Simulation time in minutes (6 weeks)
    env.run(until=simulation_time)  # Run the simulation for amount of simulation time
    
    ans = stats['total_bottles']*100-p*30*168*6-10000*6*mh

    return ans/1000

#How to use the function: bottle_filling(fbd,fsmb,lbd,lsbd,mo,frt,fsrt,lrt,lsrt,mt);
#fbd:filler breakdown rate ,fsmb: filler small breakdown rate,lbd:labeller breakdown rate,
#lsbd: labeller small breakdown rate ,mo: maintenance rate ,frt: filler big breakdown repair time,
#fsrt: filler small breakdown repair time,lrt: labeller repair time,lsrt: labeller small breakdown repair time,
#mt: maintenance time. All inputs are epxonential with the expected value being the input, except for maintenance time and
#maintenance frequency, these are a specific time between maintenance and a specific time for maintenance, since they usually
#can be determined by operations. Everything is in minutes, the output is the amount of minutes of filling line operation in 4 weeks

In [3]:
def create_single_X(k,n0):
    ll = []
    for i in range(n0):
        ll.append(bottle_filling(k[0],k[1],k[2],k[3],k[4],k[5],k[6],k[7],k[8],k[9],k[10],k[11]))
    return ll
                   
def create_X(k, n0):
    l = []
    for i in range(len(k)):        
        l.append(create_single_X(k[i],n0))
    return np.array(l)
        
def sample_variance(X):
    n = len(X[0])
    k = len(X)    
    som = 0
    for i in range(n):
        for j in range(k):
            t = np.subtract(X[j,i]-np.mean(X[:,i]),np.mean(X[j,:]))+np.mean(X)
            som += 2*t**2
    return som/(k-1)/(n-1)   


In [10]:
k1 = [24*60,60,36*60,45,24*7*60,2*60,2,4*60,4,4*60,2,4]
k2 = [24*60,60,36*60,45,24*7*60,2*60,1,4*60,2,4*60,3,4]
k3 = [48*60,60,72*60,45,24*7*60,2*60,2,4*60,4,4*60,2,8]
k4 = [72*60,60,72*60,45,24*7*60,2*60,1,4*60,2,4*60,3,8]
#k5 = [72*60,60,72*60,45,24*7*60,2*60,1,4*60,2,4*60,3,12]
kall = [k1,k2,k3,k4]

In [11]:
n_0=20

X1 = pd.DataFrame(create_X(kall,n_0))
res = pd.DataFrame({'iter': [1,2,3,4], 'pol':[2,2,3,3], 'mh':[4,8,4,8], 'mean': X1.mean(axis=1), 'var': X1.var(axis=1)})



In [12]:
d_star = 20
P_star = 0.95
h1 = 3.101

In [13]:
res['N_i'] = res.apply(lambda row: max(np.ceil((h1**2)*row["var"]/d_star**2), n_0 + 1), axis=1).astype(int)

#res.shape[0]

In [14]:
res

,iter,pol,mh,mean,var,N_i
0,1,2,4,4217.640,23189.731158,558
1,2,2,8,4402.215,21175.646605,510
2,3,3,4,4362.220,11201.223158,270
3,4,3,8,4624.335,15719.925763,378


In [15]:
res_stage2 = []

for i in range(res.shape[0]):
    h_list = create_single_X(kall[0],(res['N_i'][0]-n_0))
    res_stage2.append(np.mean(h_list))
    
# print(res_stage2)

In [16]:
res['mean(N_i - n_0)'] = res_stage2
display(res)

,iter,pol,mh,mean,var,N_i,mean(N_i - n_0)
0,1,2,4,4217.640,23189.731158,558,4228.362565
1,2,2,8,4402.215,21175.646605,510,4219.402156
2,3,3,4,4362.220,11201.223158,270,4219.877621
3,4,3,8,4624.335,15719.925763,378,4225.966097


In [17]:
resm = res.copy()

In [19]:
resm['square_brac'] = 1-(resm['N_i']-n_0)*d_star**2/(h1**2*resm['var'])
resm['square_root_arg'] = 1 - resm['N_i']/n_0*resm['square_brac'] 
resm['square_root'] = np.sqrt(resm['square_root_arg'])
resm['W_i1'] = (n_0/resm['N_i'])*(1+resm['square_root'])
resm['W_i2'] = 1-resm['W_i1']
resm['X_tilda'] = (resm['W_i1']*resm['mean'] + resm['W_i2']*resm['mean(N_i - n_0)'])
resm = resm.drop(['square_brac', 'square_root_arg','square_root'], axis=1)
display(resm)

,iter,pol,mh,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,1,2,4,4217.640,23189.731158,558,4228.362565,0.041449,0.958551,4227.918120
1,2,2,8,4402.215,21175.646605,510,4219.402156,0.047498,0.952502,4228.085392
2,3,3,4,4362.220,11201.223158,270,4219.877621,0.087588,0.912412,4232.345037
3,4,3,8,4624.335,15719.925763,378,4225.966097,0.056269,0.943731,4248.381892


In [20]:
resm['mean(N_i - n_0)'] = resm['mean(N_i - n_0)']
resm['X_tilda'] = resm['X_tilda']

In [21]:
resm.drop(columns = ['iter'])

,pol,mh,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,2,4,4217.640,23189.731158,558,4228.362565,0.041449,0.958551,4227.918120
1,2,8,4402.215,21175.646605,510,4219.402156,0.047498,0.952502,4228.085392
2,3,4,4362.220,11201.223158,270,4219.877621,0.087588,0.912412,4232.345037
3,3,8,4624.335,15719.925763,378,4225.966097,0.056269,0.943731,4248.381892


In [22]:
print(resm.drop(columns = ['iter']).to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.4f}".format))

\begin{tabular}{lrrrrrrrrr}
\toprule
{} &  pol &  mh &      mean &        var &  N\_i &  mean(N\_i - n\_0) &   W\_i1 &   W\_i2 &   X\_tilda \\
\midrule
0 &    2 &   4 & 4217.6400 & 23189.7312 &  558 &        4228.3626 & 0.0414 & 0.9586 & 4227.9181 \\
1 &    2 &   8 & 4402.2150 & 21175.6466 &  510 &        4219.4022 & 0.0475 & 0.9525 & 4228.0854 \\
2 &    3 &   4 & 4362.2200 & 11201.2232 &  270 &        4219.8776 & 0.0876 & 0.9124 & 4232.3450 \\
3 &    3 &   8 & 4624.3350 & 15719.9258 &  378 &        4225.9661 & 0.0563 & 0.9437 & 4248.3819 \\
\bottomrule
\end{tabular}

